In [1]:
import pandas as pd   # package for data analysis
import numpy as np    # package for numerical computations

# libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_csv("Train.csv")
train_data.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [3]:
train_data.isnull().sum()

Store_Ratio       0
Basket_Ratio      0
Category_1        0
Store_Score       0
Category_2        0
Store_Presence    0
Score_1           0
Score_2           0
Score_3           0
Score_4           0
time              0
popularity        0
dtype: int64

In [4]:
train_data.describe()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
count,18208.000000,18208.000000,18208.000000,18208.000000,18208.000000,18208.000000,18208.000000,18208.000000,18208.000000,18208.000000,1.820800e+04,18208.000000
mean,0.544283,0.483585,5.155536,-12.198086,0.648506,0.477702,0.322109,0.164888,0.421440,115.305776,2.180101e+05,3.850231
std,0.202709,0.302010,3.535068,8.370566,0.477450,0.380634,0.413493,0.136531,0.271922,31.478303,9.648289e+04,0.544226
min,0.000000,0.000216,0.000000,-47.576000,0.000000,0.000000,0.000000,0.011900,0.000000,0.000000,3.184400e+04,0.000000
25%,0.411000,0.200000,2.000000,-16.496250,0.000000,0.086175,0.000001,0.095300,0.184750,90.974250,1.696202e+05,4.000000
50%,0.573000,0.517000,5.000000,-9.166500,1.000000,0.430000,0.002245,0.112000,0.393000,114.022500,2.046460e+05,4.000000
75%,0.699000,0.742000,8.000000,-5.943750,1.000000,0.895000,0.859000,0.176000,0.640000,134.997000,2.471562e+05,4.000000
max,0.998000,1.000000,11.000000,-0.079000,1.000000,0.996000,1.000000,0.991000,0.999000,219.701000,3.908467e+06,5.000000


In [5]:
# For Preprocessing, ML models and Evaluation
from sklearn.model_selection import train_test_split   # To split the dataset into train and test set

In [7]:
X = train_data.drop('popularity', axis=1)
y = train_data['popularity']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [56]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.metrics import log_loss

In [22]:
test_data = pd.read_csv('Test.csv')

In [26]:
### Logistic Regression
logr = LogisticRegression()
logr.fit(X_train, y_train)
y_pred_lr = logr.predict(X_test)
logr_train_acc = round(logr.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(logr_train_acc) + '%')

logr_test_acc = round(logr.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(logr_test_acc) + '%')

Train Accuracy: 83.21%
Test Accuracy: 83.88%


In [68]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

folds = StratifiedKFold(n_splits=5,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,y)):
  train_set = (X.iloc[train_idx],y.iloc[train_idx])
  test_set = (X.iloc[test_idx],y.iloc[test_idx])

  model = CatBoostClassifier(iterations=9, learning_rate=0.9)
  model.fit(*train_set)

  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test_data))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0:	learn: 0.5501462	total: 39.6ms	remaining: 317ms
1:	learn: 0.5032092	total: 61.2ms	remaining: 214ms
2:	learn: 0.4843661	total: 77.9ms	remaining: 156ms
3:	learn: 0.4728888	total: 94.8ms	remaining: 118ms
4:	learn: 0.4653968	total: 110ms	remaining: 88.1ms
5:	learn: 0.4596782	total: 126ms	remaining: 62.8ms
6:	learn: 0.4542693	total: 142ms	remaining: 40.5ms
7:	learn: 0.4500354	total: 157ms	remaining: 19.6ms
8:	learn: 0.4451867	total: 173ms	remaining: 0us
0.4918681989674427
0:	learn: 0.5516789	total: 18.5ms	remaining: 148ms
1:	learn: 0.5050991	total: 35ms	remaining: 123ms
2:	learn: 0.4827773	total: 50.5ms	remaining: 101ms
3:	learn: 0.4711014	total: 66.3ms	remaining: 82.9ms
4:	learn: 0.4618745	total: 81.4ms	remaining: 65.1ms
5:	learn: 0.4580706	total: 97.4ms	remaining: 48.7ms
6:	learn: 0.4540813	total: 113ms	remaining: 32.1ms
7:	learn: 0.4495259	total: 130ms	remaining: 16.2ms
8:	learn: 0.4439888	total: 150ms	remaining: 0us
0.4826555761451861
0:	learn: 0.5554136	total: 21.1ms	remaining: 169m

In [61]:
submission = pd.DataFrame(final_preds)
submission.to_csv("submission.csv",index=None)
submission

,0,1,2,3,4
0,0.008543,0.015156,0.092459,0.865479,0.018363
1,0.008714,0.015339,0.071583,0.887093,0.017271
2,0.008447,0.014997,0.060629,0.900263,0.015663
3,0.008542,0.015590,0.063203,0.896885,0.015780
4,0.008469,0.014651,0.061767,0.898489,0.016623
...,...,...,...,...,...
12135,0.008429,0.016101,0.147056,0.809043,0.019370
12136,0.008469,0.014651,0.061767,0.898489,0.016623
12137,0.008477,0.014833,0.063129,0.896959,0.016602
12138,0.008837,0.015972,0.069106,0.887789,0.018297


In [10]:
### Support Vector
clf = SVC()
clf.fit(X_train, y_train)
y_pred_svc = clf.predict(X_test)
acc_svc = round(clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_svc) + '%')

acc_svc = round(clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_svc) + '%')

Train Accuracy: 83.56%
Test Accuracy: 84.02%


In [11]:
# Linear SVC
clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred_linear_svc = clf.predict(X_test)
acc_linear_svc = round(clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_linear_svc) + '%')

acc_linear_svc = round(clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_linear_svc) + '%')

Train Accuracy: 83.56%
Test Accuracy: 84.02%


In [12]:
# KN Neighbors
kn_clf = KNeighborsClassifier(n_neighbors = 2)
kn_clf.fit(X_train, y_train)
y_pred_knn = kn_clf.predict(X_test)
acc_knn = round(kn_clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_knn) + '%')

acc_knn = round(kn_clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_knn) + '%')

Train Accuracy: 90.86%
Test Accuracy: 74.24%


In [13]:
# Decision Tree Classifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
acc_dt = round(dt_clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_dt) + '%')

acc_knn = round(kn_clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_knn) + '%')

Train Accuracy: 100.0%
Test Accuracy: 74.24%


In [14]:
# Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
acc_rf = round(rf_clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_rf) + '%')

acc_rf = round(rf_clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_rf) + '%')

Train Accuracy: 100.0%
Test Accuracy: 87.67%


In [15]:
# GaussionNB Classifier
gb_clf = GaussianNB()
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
acc_gb = round(gb_clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_gb) + '%')

acc_gb = round(rf_clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_gb) + '%')

Train Accuracy: 81.81%
Test Accuracy: 87.67%


In [16]:
# CatBoost Classification
from catboost import CatBoostClassifier
cb_clf = CatBoostClassifier()
cb_clf.fit(X_train, y_train)
y_pred_cb = cb_clf.predict(X_test)
acc_cb = round(cb_clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_cb) + '%')

acc_cb = round(cb_clf.score(X_test, y_test) * 100, 2)
print ("Test Accuracy: " + str(acc_cb) + '%')

Learning rate set to 0.09071
0:	learn: 1.3794771	total: 624ms	remaining: 10m 23s
1:	learn: 1.2251994	total: 687ms	remaining: 5m 42s
2:	learn: 1.1117301	total: 722ms	remaining: 3m 59s
3:	learn: 1.0210454	total: 759ms	remaining: 3m 8s
4:	learn: 0.9479464	total: 792ms	remaining: 2m 37s
5:	learn: 0.8873345	total: 823ms	remaining: 2m 16s
6:	learn: 0.8357505	total: 854ms	remaining: 2m 1s
7:	learn: 0.7923198	total: 889ms	remaining: 1m 50s
8:	learn: 0.7552956	total: 923ms	remaining: 1m 41s
9:	learn: 0.7232013	total: 948ms	remaining: 1m 33s
10:	learn: 0.6950692	total: 980ms	remaining: 1m 28s
11:	learn: 0.6704560	total: 1.01s	remaining: 1m 23s
12:	learn: 0.6488679	total: 1.04s	remaining: 1m 18s
13:	learn: 0.6308533	total: 1.06s	remaining: 1m 14s
14:	learn: 0.6140782	total: 1.1s	remaining: 1m 11s
15:	learn: 0.5997666	total: 1.13s	remaining: 1m 9s
16:	learn: 0.5868146	total: 1.16s	remaining: 1m 6s
17:	learn: 0.5749680	total: 1.19s	remaining: 1m 5s
18:	learn: 0.5642227	total: 1.23s	remaining: 1m 3s

163:	learn: 0.3972807	total: 5.47s	remaining: 27.9s
164:	learn: 0.3968700	total: 5.51s	remaining: 27.9s
165:	learn: 0.3962798	total: 5.54s	remaining: 27.8s
166:	learn: 0.3958240	total: 5.57s	remaining: 27.8s
167:	learn: 0.3952436	total: 5.6s	remaining: 27.7s
168:	learn: 0.3948085	total: 5.63s	remaining: 27.7s
169:	learn: 0.3944622	total: 5.66s	remaining: 27.7s
170:	learn: 0.3940865	total: 5.69s	remaining: 27.6s
171:	learn: 0.3937304	total: 5.72s	remaining: 27.5s
172:	learn: 0.3934069	total: 5.75s	remaining: 27.5s
173:	learn: 0.3930920	total: 5.78s	remaining: 27.4s
174:	learn: 0.3927970	total: 5.81s	remaining: 27.4s
175:	learn: 0.3924690	total: 5.84s	remaining: 27.3s
176:	learn: 0.3920581	total: 5.87s	remaining: 27.3s
177:	learn: 0.3915356	total: 5.9s	remaining: 27.3s
178:	learn: 0.3911278	total: 5.93s	remaining: 27.2s
179:	learn: 0.3907068	total: 5.96s	remaining: 27.1s
180:	learn: 0.3901130	total: 5.99s	remaining: 27.1s
181:	learn: 0.3898478	total: 6.01s	remaining: 27s
182:	learn: 0.38

323:	learn: 0.3407136	total: 10.1s	remaining: 21.1s
324:	learn: 0.3403446	total: 10.1s	remaining: 21.1s
325:	learn: 0.3399980	total: 10.2s	remaining: 21s
326:	learn: 0.3395185	total: 10.2s	remaining: 21s
327:	learn: 0.3392230	total: 10.2s	remaining: 21s
328:	learn: 0.3387849	total: 10.3s	remaining: 20.9s
329:	learn: 0.3385513	total: 10.3s	remaining: 20.9s
330:	learn: 0.3383267	total: 10.3s	remaining: 20.9s
331:	learn: 0.3378033	total: 10.4s	remaining: 20.8s
332:	learn: 0.3376086	total: 10.4s	remaining: 20.8s
333:	learn: 0.3372409	total: 10.4s	remaining: 20.8s
334:	learn: 0.3369013	total: 10.4s	remaining: 20.7s
335:	learn: 0.3365946	total: 10.5s	remaining: 20.7s
336:	learn: 0.3362776	total: 10.5s	remaining: 20.7s
337:	learn: 0.3360741	total: 10.5s	remaining: 20.6s
338:	learn: 0.3357980	total: 10.6s	remaining: 20.6s
339:	learn: 0.3355395	total: 10.6s	remaining: 20.6s
340:	learn: 0.3352725	total: 10.6s	remaining: 20.5s
341:	learn: 0.3351189	total: 10.7s	remaining: 20.5s
342:	learn: 0.3349

486:	learn: 0.2984002	total: 14.9s	remaining: 15.7s
487:	learn: 0.2981499	total: 15s	remaining: 15.7s
488:	learn: 0.2978995	total: 15s	remaining: 15.7s
489:	learn: 0.2976301	total: 15s	remaining: 15.6s
490:	learn: 0.2974334	total: 15s	remaining: 15.6s
491:	learn: 0.2972677	total: 15.1s	remaining: 15.6s
492:	learn: 0.2970716	total: 15.1s	remaining: 15.5s
493:	learn: 0.2969037	total: 15.1s	remaining: 15.5s
494:	learn: 0.2967612	total: 15.2s	remaining: 15.5s
495:	learn: 0.2964262	total: 15.2s	remaining: 15.4s
496:	learn: 0.2961237	total: 15.2s	remaining: 15.4s
497:	learn: 0.2959529	total: 15.3s	remaining: 15.4s
498:	learn: 0.2957579	total: 15.3s	remaining: 15.4s
499:	learn: 0.2955037	total: 15.3s	remaining: 15.3s
500:	learn: 0.2952523	total: 15.4s	remaining: 15.3s
501:	learn: 0.2949644	total: 15.4s	remaining: 15.3s
502:	learn: 0.2947733	total: 15.4s	remaining: 15.3s
503:	learn: 0.2945297	total: 15.5s	remaining: 15.2s
504:	learn: 0.2941823	total: 15.5s	remaining: 15.2s
505:	learn: 0.294044

650:	learn: 0.2641508	total: 19.8s	remaining: 10.6s
651:	learn: 0.2639165	total: 19.8s	remaining: 10.6s
652:	learn: 0.2638159	total: 19.8s	remaining: 10.5s
653:	learn: 0.2635964	total: 19.9s	remaining: 10.5s
654:	learn: 0.2634345	total: 19.9s	remaining: 10.5s
655:	learn: 0.2632086	total: 19.9s	remaining: 10.5s
656:	learn: 0.2630392	total: 20s	remaining: 10.4s
657:	learn: 0.2629218	total: 20s	remaining: 10.4s
658:	learn: 0.2625906	total: 20s	remaining: 10.4s
659:	learn: 0.2623835	total: 20.1s	remaining: 10.3s
660:	learn: 0.2622365	total: 20.1s	remaining: 10.3s
661:	learn: 0.2620069	total: 20.1s	remaining: 10.3s
662:	learn: 0.2617932	total: 20.2s	remaining: 10.2s
663:	learn: 0.2616002	total: 20.2s	remaining: 10.2s
664:	learn: 0.2614092	total: 20.2s	remaining: 10.2s
665:	learn: 0.2612355	total: 20.2s	remaining: 10.1s
666:	learn: 0.2610372	total: 20.3s	remaining: 10.1s
667:	learn: 0.2609527	total: 20.3s	remaining: 10.1s
668:	learn: 0.2606405	total: 20.3s	remaining: 10.1s
669:	learn: 0.2603

813:	learn: 0.2349798	total: 24.7s	remaining: 5.64s
814:	learn: 0.2346979	total: 24.7s	remaining: 5.61s
815:	learn: 0.2345271	total: 24.7s	remaining: 5.57s
816:	learn: 0.2343143	total: 24.8s	remaining: 5.54s
817:	learn: 0.2341049	total: 24.8s	remaining: 5.51s
818:	learn: 0.2339715	total: 24.8s	remaining: 5.48s
819:	learn: 0.2337065	total: 24.8s	remaining: 5.45s
820:	learn: 0.2336359	total: 24.9s	remaining: 5.42s
821:	learn: 0.2335313	total: 24.9s	remaining: 5.4s
822:	learn: 0.2333095	total: 25s	remaining: 5.37s
823:	learn: 0.2332303	total: 25s	remaining: 5.34s
824:	learn: 0.2330485	total: 25s	remaining: 5.31s
825:	learn: 0.2328856	total: 25.1s	remaining: 5.28s
826:	learn: 0.2327405	total: 25.1s	remaining: 5.25s
827:	learn: 0.2325320	total: 25.1s	remaining: 5.22s
828:	learn: 0.2324588	total: 25.1s	remaining: 5.19s
829:	learn: 0.2323468	total: 25.2s	remaining: 5.16s
830:	learn: 0.2322248	total: 25.2s	remaining: 5.13s
831:	learn: 0.2320989	total: 25.2s	remaining: 5.09s
832:	learn: 0.23185

975:	learn: 0.2108360	total: 29.5s	remaining: 727ms
976:	learn: 0.2106966	total: 29.6s	remaining: 696ms
977:	learn: 0.2106134	total: 29.6s	remaining: 666ms
978:	learn: 0.2104485	total: 29.6s	remaining: 636ms
979:	learn: 0.2104081	total: 29.7s	remaining: 605ms
980:	learn: 0.2102567	total: 29.7s	remaining: 575ms
981:	learn: 0.2101723	total: 29.7s	remaining: 545ms
982:	learn: 0.2099318	total: 29.8s	remaining: 514ms
983:	learn: 0.2098630	total: 29.8s	remaining: 484ms
984:	learn: 0.2096638	total: 29.8s	remaining: 454ms
985:	learn: 0.2095346	total: 29.8s	remaining: 424ms
986:	learn: 0.2094694	total: 29.9s	remaining: 393ms
987:	learn: 0.2093450	total: 29.9s	remaining: 363ms
988:	learn: 0.2092098	total: 29.9s	remaining: 333ms
989:	learn: 0.2091371	total: 30s	remaining: 303ms
990:	learn: 0.2090486	total: 30s	remaining: 272ms
991:	learn: 0.2089249	total: 30s	remaining: 242ms
992:	learn: 0.2087860	total: 30s	remaining: 212ms
993:	learn: 0.2087014	total: 30.1s	remaining: 182ms
994:	learn: 0.208649